In [1]:
import os
os.getcwd()

'/home/yes24/capstone/추천알고리즘/preprocessed'

In [2]:
os.chdir('/home/yes24/models/research/deep_contextual_bandits')

In [3]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import pandas as pd
import tensorflow as tf
import time
from absl import app
from absl import flags

In [4]:
from bandits.core.contextual_bandit import run_contextual_bandit
from bandits.algorithms.linear_full_posterior_sampling import LinearFullPosteriorSampling

In [5]:
from bandits.data.data_sampler import sample_census_data
from bandits.data.data_sampler import one_hot,safe_std,remove_underrepresented_classes,classification_to_bandit_problem

In [6]:
def display_results(algos, opt_rewards, opt_actions, h_rewards, t_init, name):
    """Displays summary statistics of the performance of each algorithm."""

    print('---------------------------------------------------')
    print('---------------------------------------------------')
    print('{} bandit completed after {} seconds.'.format(
        name, time.time() - t_init))
    print('---------------------------------------------------')

    performance_pairs = []
    for j, a in enumerate(algos):
        performance_pairs.append((a.name, np.sum(h_rewards[:, j])))
    performance_pairs = sorted(performance_pairs,
                             key=lambda elt: elt[1],
                             reverse=True)
    for i, (name, reward) in enumerate(performance_pairs):
        print('{:3}) {:20}| \t \t total reward = {:10}.'.format(i, name, reward))

    print('---------------------------------------------------')
    print('Optimal total reward = {}.'.format(np.sum(opt_rewards)))
    print('Frequency of optimal actions (action, frequency):')
    print([[elt, list(opt_actions).count(elt)] for elt in set(opt_actions)])
    print('---------------------------------------------------')
    print('---------------------------------------------------')



In [7]:
num_actions = 5

In [8]:
df_100_fin = pd.read_csv('df100_test_0701.csv')

In [9]:
df_100_fin['purchase'].sum()

706.0

In [10]:
df_context = df_100_fin.iloc[:,6:]
labels = df_100_fin.iloc[:,4].astype('int8').values

In [11]:
df_context

,gender_0.0,gender_1.0,g_age_0.0,g_age_1.0,g_age_2.0,g_age_3.0,g_age_4.0,g_age_5.0,g_age_6.0,most_pref_1.0,...,cat34.0,pub0.0,pub1.0,pub2.0,pub3.0,pub4.0,pri0.0,pri1.0,pri2.0,pri3.0
0,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,1,0
1,1,0,0,0,0,0,1,0,0,0,...,1,0,1,0,0,0,0,0,1,0
2,1,0,0,0,0,0,1,0,0,0,...,1,0,0,1,0,0,0,0,1,0
3,1,0,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,1,0,0
4,1,0,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4128,0,1,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,1,0
4129,0,1,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,1,0
4130,0,1,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,1,0
4131,0,1,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,1,0


In [12]:
df_context, labels = remove_underrepresented_classes(df_context, labels)
contexts = df_context.values

Unique classes and their ratio of total: {0: 0.0931526736027099, 1: 0.18872489716912655, 2: 0.17735301233970482, 3: 0.432615533510767, 4: 0.10815388337769175}


In [13]:
sampled_vals = classification_to_bandit_problem(contexts, labels, num_actions)

In [14]:
contexts, rewards, (opt_rewards, opt_actions) = sampled_vals

In [15]:
rewards_list = df_100_fin['purchase'].values.tolist()

In [16]:
rewards2 = rewards.copy()

In [17]:
for i in range(len(rewards_list)):
    if rewards_list[i] == 0:
        rewards2[i] = np.where(rewards2[i]==1,0,rewards2[i])

In [18]:
dataset = np.hstack((contexts, rewards))
context_dim = contexts.shape[1]

In [19]:
dataset2 = np.hstack((contexts, rewards2))
context_dim = contexts.shape[1]

In [20]:
mu = np.load('/home/yes24/capstone/소라공유폴더/mean_0701.npy')
cov = np.load('/home/yes24/capstone/소라공유폴더/cov_0701.npy')

In [21]:
hparams = tf.contrib.training.HParams(num_actions=num_actions)
hparams_linear = tf.contrib.training.HParams(num_actions=num_actions,
                                               context_dim=context_dim,
                                               a0=6,
                                               b0=6,
                                               lambda_prior=0.25,
                                               initial_pulls=0,
                                            mu=mu,
                                            cov=cov)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [22]:
algos = [LinearFullPosteriorSampling('LinFullPost', hparams_linear)]

In [23]:
t_init2 = time.time()
results2 = run_contextual_bandit(context_dim, num_actions, dataset2, algos)


In [24]:
_2, h_rewards2,h_vals2 = results2

In [25]:
len(h_vals2)

4133

In [26]:
h_vals2[0][0]

[-10.897289707237338,
 10.568306609291731,
 11.46056392596493,
 7.934488286205957,
 -11.453784921234451]

In [27]:
new_vals = [val[0] for val in h_vals2]

In [28]:
len(new_vals)

4133

In [29]:
val_df = pd.DataFrame()
for i in range(len(new_vals)):
    val_df = pd.concat([val_df,pd.DataFrame(new_vals[i]).T],axis=0)

In [30]:
val_df['val_max'] = val_df[[0,1,2,3,4]].max(axis=1)

In [34]:
val_df_fin = val_df.reset_index().drop(columns='index')

In [51]:
df_100_fin

,account_id,product_id,price,purchase,book_cluster,user_cluster,gender_0.0,gender_1.0,g_age_0.0,g_age_1.0,...,cat34.0,pub0.0,pub1.0,pub2.0,pub3.0,pub4.0,pri0.0,pri1.0,pri2.0,pri3.0
0,113512,89940862,0.0,0.0,4.0,3.0,1,0,0,0,...,0,0,0,0,0,1,0,0,1,0
1,113512,84434768,0.0,0.0,2.0,3.0,1,0,0,0,...,1,0,1,0,0,0,0,0,1,0
2,113512,74393338,0.0,0.0,2.0,3.0,1,0,0,0,...,1,0,0,1,0,0,0,0,1,0
3,113512,90108425,0.0,0.0,3.0,3.0,1,0,0,0,...,0,1,0,0,0,0,0,1,0,0
4,113512,85927898,0.0,0.0,3.0,3.0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4128,16790454,90061659,0.0,0.0,3.0,1.0,0,1,0,0,...,0,1,0,0,0,0,0,0,1,0
4129,16790454,89222585,14400.0,1.0,3.0,1.0,0,1,0,0,...,0,1,0,0,0,0,0,0,1,0
4130,16790454,89903437,0.0,0.0,3.0,1.0,0,1,0,0,...,0,1,0,0,0,0,0,0,1,0
4131,16790454,90114596,0.0,0.0,3.0,1.0,0,1,0,0,...,0,1,0,0,0,0,0,0,1,0


In [36]:
df_fin_fin = pd.concat([df_100_fin[['account_id','product_id']],val_df_fin],axis=1)

In [43]:
account_list = df_fin_fin['account_id'].unique()

In [60]:
recc_list = []
for i in range(len(account_list)):
    recc_list.append([df_fin_fin[df_fin_fin['account_id']==account_list[i]].sort_values('val_max',ascending=False)['product_id'].tolist()[0:24]])

In [64]:
recc_list[0][0]

[85927898, 74393338, 84434768, 89940862, 90108425]

In [70]:
recc_list

[[[85927898, 74393338, 84434768, 89940862, 90108425]],
 [[61941573,
   34919942,
   44023785,
   79299806,
   86939351,
   78509413,
   85474822,
   34919942,
   44331788,
   34919942,
   85927898,
   34919942,
   44022506,
   67292643,
   85927898,
   57913235,
   67292661,
   34919948,
   44331788,
   44023785,
   67292643,
   61941565]],
 [[32235690,
   90114596,
   73621198,
   66796730,
   90114596,
   85927898,
   89938195,
   90061659,
   46574892,
   85927898,
   64341011,
   67151257,
   59677271,
   89928090,
   60569960,
   90002606,
   79264366,
   85578260,
   89308816,
   89402159,
   83549800,
   79264366,
   85578260]],
 [[3173899,
   89831765,
   20289045,
   85927898,
   65282018,
   81524314,
   89869527,
   20289045,
   89707566,
   89707566]],
 [[24912652,
   86895523,
   89904068,
   24259730,
   90082148,
   24259730,
   24912652,
   85927898,
   90039222,
   89851054]],
 [[37120970,
   15280734,
   26889874,
   89869475,
   90040990,
   9163765,
   28325977,
   

In [69]:
[len(recc[0]) for recc in recc_list]

[5,
 22,
 23,
 10,
 10,
 16,
 9,
 3,
 24,
 24,
 5,
 24,
 24,
 9,
 24,
 24,
 24,
 19,
 24,
 24,
 24,
 24,
 14,
 24,
 24,
 24,
 18,
 14,
 24,
 7,
 9,
 24,
 24,
 24,
 15,
 7,
 4,
 24,
 24,
 24,
 24,
 3,
 15,
 24,
 21,
 24,
 1,
 21,
 2,
 8,
 12,
 19,
 9,
 17,
 24,
 11,
 5,
 12,
 24,
 24,
 10,
 8,
 23,
 3,
 24,
 23,
 24,
 24,
 2,
 24,
 12,
 24,
 12,
 7,
 14,
 17,
 24,
 24,
 24,
 15,
 12,
 24,
 21,
 18,
 14,
 16,
 10,
 11,
 12,
 24,
 11,
 4,
 24,
 4,
 7,
 24,
 24,
 24,
 4,
 24]

In [75]:
recc_book_list = [y for x in recc_book_list for y in x]

In [81]:
recc_book_list = list(set(recc_book_list))

In [85]:
recc_book_list.remove(85927898)

In [89]:
recc_book_list


[88389638,
 37142534,
 6318087,
 69734406,
 85030922,
 29653007,
 22491153,
 75552785,
 2312211,
 2664470,
 59672599,
 89913366,
 62267420,
 22865949,
 89894948,
 89317414,
 90036265,
 60526635,
 58603565,
 241709,
 69310511,
 40560690,
 17561653,
 45512761,
 90087484,
 3342397,
 89894977,
 89987138,
 90075205,
 87631944,
 2709577,
 89827405,
 89927758,
 16777296,
 74031185,
 90075216,
 28325977,
 12404840,
 86241389,
 89581677,
 89327727,
 89987184,
 71997555,
 89903225,
 12648570,
 87629946,
 75554940,
 88373374,
 90003585,
 44179587,
 28113033,
 1806475,
 71954572,
 78217357,
 29190285,
 62095502,
 90075280,
 11546769,
 38441106,
 75235476,
 33581209,
 86608026,
 2183322,
 88371357,
 60778656,
 90075298,
 89553061,
 89553062,
 32235690,
 12687530,
 83781804,
 19114166,
 30152890,
 90075325,
 83519680,
 2752705,
 80457923,
 89649356,
 90085586,
 37525716,
 5085397,
 90075357,
 32536800,
 90075364,
 74602725,
 89221351,
 34914538,
 31076595,
 78651636,
 67479801,
 65067259,
 72159493,

In [93]:
df_100_fin[df_100_fin['product_id'].isin(recc_book_list)]['purchase'].sum()

472.0

In [57]:
df_fin_fin[df_fin_fin['account_id']==account_list[0]].sort_values('val_max',ascending=False)['product_id'].tolist()[0:24]

[85927898, 74393338, 84434768, 89940862, 90108425]

In [ ]:
df_fin_fin[df_fin_fin['account_id']==account_list[0]].sort_values('val_max',ascending=False)['product_id']




In [41]:
df_fin_fin.sort_values(['account_id','val_max'],ascending=False)[0:50]

,account_id,product_id,0,1,2,3,4,val_max
4120,16790454,37300128,-0.086427,0.499775,0.050252,-0.174359,-0.039236,0.499775
4119,16790454,79297023,-0.024507,0.482987,0.001415,-0.151331,-0.142064,0.482987
4117,16790454,37300128,-0.133613,0.455308,-0.102648,-0.241132,0.043122,0.455308
4124,16790454,37300128,-0.041165,0.447951,-0.060858,-0.287709,0.044051,0.447951
4108,16790454,37300128,-0.132805,0.393513,-0.038143,-0.239686,-0.025957,0.393513
4103,16790454,37300128,-0.094839,0.323402,-0.048594,-0.128818,-0.032212,0.323402
4088,16790454,79297023,-0.069809,0.323316,-0.139896,-0.148668,-0.184082,0.323316
4073,16790454,79297023,-0.046853,0.263619,0.118237,-0.419719,-0.022967,0.263619
4072,16790454,59260220,0.112732,-0.282695,0.255865,-0.229528,0.124067,0.255865
4100,16790454,37300128,-0.028108,0.239709,-0.169376,-0.313396,-0.085323,0.239709


In [96]:
t_init = time.time()
results = run_contextual_bandit(context_dim, num_actions, dataset, algos)


ValueError: too many values to unpack (expected 2)

In [97]:
_, h_rewards,h_vals = results

In [98]:
display_results(algos, opt_rewards, opt_actions, h_rewards, t_init, 'yes24')

---------------------------------------------------
---------------------------------------------------
yes24 bandit completed after 172.4958164691925 seconds.
---------------------------------------------------
  0) LinFullPost         | 	 	 total reward =     3495.0.
---------------------------------------------------
Optimal total reward = 4133.0.
Frequency of optimal actions (action, frequency):
[[0, 385], [1, 780], [2, 733], [3, 1788], [4, 447]]
---------------------------------------------------
---------------------------------------------------


In [94]:
display_results(algos, opt_rewards, opt_actions, h_rewards2, t_init2, 'yes24')

---------------------------------------------------
---------------------------------------------------
yes24 bandit completed after 1718.75066781044 seconds.
---------------------------------------------------
  0) LinFullPost         | 	 	 total reward =      381.0.
---------------------------------------------------
Optimal total reward = 4133.0.
Frequency of optimal actions (action, frequency):
[[0, 385], [1, 780], [2, 733], [3, 1788], [4, 447]]
---------------------------------------------------
---------------------------------------------------


In [31]:
display_results(algos, opt_rewards, opt_actions, h_rewards2, t_init2, 'yes24')

---------------------------------------------------
---------------------------------------------------
yes24 bandit completed after 93.9321277141571 seconds.
---------------------------------------------------
  0) LinFullPost         | 	 	 total reward =       82.0.
---------------------------------------------------
Optimal total reward = 4133.0.
Frequency of optimal actions (action, frequency):
[[0, 385], [1, 780], [2, 733], [3, 1788], [4, 447]]
---------------------------------------------------
---------------------------------------------------


In [31]:
ts_actions = _.T[0]

In [36]:
len(ts_actions)

4133

In [37]:
len(labels)

4133

In [40]:
len(rewards_list)

4133

In [45]:
rewards_list = np.array(rewards_list)

In [50]:
df_eval = pd.concat([pd.DataFrame(ts_actions),pd.DataFrame(labels),pd.DataFrame(rewards_list)],axis=1)

In [52]:
df_eval.columns=['ts_actions','labels','reward']

In [60]:
rewards_fin = 0
for i in range(len(df_eval)):
    if df_eval['reward'][i] == 1:
        if df_eval['ts_actions'][i] == df_eval['labels'][i]:
            rewards_fin +=1

In [61]:
rewards_fin

615

In [68]:
mean1 = algos[0].mu

In [69]:
cov1 = algos[0].cov

In [75]:
mean2 = algos[0].mu

In [76]:
cov2 = algos[0].cov

In [31]:
algos[0].mu

[array([ 2.94238351e-03, -2.94238351e-03,  0.00000000e+00,  1.86575911e-02,
        -2.70227944e-02, -1.15259194e-02,  2.31228639e-02,  5.91907131e-03,
         0.00000000e+00, -2.09351004e-02,  5.09044305e-03, -2.21060517e-02,
        -1.42324452e-02,  5.62984922e-05, -2.10951052e-02,  2.05857716e-03,
         5.08096442e-03, -8.19246790e-03,  1.84492896e-02,  0.00000000e+00,
         1.05237238e-02,  1.97907673e-02,  0.00000000e+00, -2.15151753e-02,
        -1.46805238e-02, -1.08972048e-02, -4.87409864e-03, -1.37862495e-02,
         2.45011192e-02, -6.91920123e-03,  2.83958830e-02, -2.15381536e-03,
         9.58653860e-03, -4.11546726e-03, -1.32562954e-03,  1.32562954e-03,
         1.69998275e-02, -1.69998275e-02, -1.48153127e-03,  1.00725578e-02,
         8.28318908e-02, -1.44173775e-02,  3.27791961e-02,  3.81069300e-02,
         1.08810215e-02,  1.08113050e-02, -9.18157155e-03,  3.27987480e-02,
         1.06950729e-02,  2.76052955e-02,  5.91797466e-02,  6.10902328e-03,
         3.1

In [8]:
df_100 = pd.read_csv('/home/yes24/capstone/영송공유폴더/test_100.csv')
df_100

,account_id,product_id,price,purchase,book_cluster,user_cluster
0,113512,89940862,0.0,0.0,4.0,3.0
1,113512,84434768,0.0,0.0,2.0,3.0
2,113512,74393338,0.0,0.0,2.0,3.0
3,113512,90108425,0.0,0.0,3.0,3.0
4,113512,85927898,0.0,0.0,3.0,3.0
...,...,...,...,...,...,...
4128,16790454,90061659,0.0,0.0,3.0,1.0
4129,16790454,89222585,14400.0,1.0,3.0,1.0
4130,16790454,89903437,0.0,0.0,3.0,1.0
4131,16790454,90114596,0.0,0.0,3.0,1.0


In [13]:
user_fin = pd.read_csv('/home/yes24/capstone/원진공유폴더/일요일에고생이많다/user_dummified_1df_0628.csv')

In [14]:
book_fin = pd.read_csv('/home/yes24/capstone/원진공유폴더/일요일에고생이많다/bookfeature_dummified_0628.csv')
book_fin.drop(columns=book_fin.columns[0],inplace=True)

In [15]:
user100_list = df_100['account_id'].unique()

In [16]:
pro100_list = df_100['product_id'].unique()

In [17]:
user_fin[user_fin['account_id'].isin(user100_list)]

,account_id,gender_0.0,gender_1.0,g_age_0.0,g_age_1.0,g_age_2.0,g_age_3.0,g_age_4.0,g_age_5.0,g_age_6.0,...,most_pref_23.0,most_pref_25.0,most_pref_27.0,most_pref_29.0,most_pref_33.0,most_pref_34.0,new_pref_0,new_pref_1,address_is_0,address_is_1
12301,231022.0,1,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,1,0,0,1
13217,2232676.0,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,1,1,0,0,1
13641,3300415.0,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,1
28128,9805328.0,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,1
49539,2122933.0,1,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1166997,1569435.0,0,1,0,0,0,0,1,0,0,...,0,0,1,0,0,0,1,0,0,1
1167226,2332587.0,0,1,0,0,0,0,1,0,0,...,0,0,0,0,1,0,1,0,0,1
1204153,3056585.0,1,0,0,0,0,0,1,0,0,...,0,0,0,1,0,0,1,0,0,1
1209537,1176113.0,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,1,1,0,0,1


In [18]:
book_fin[book_fin['product_id'].isin(pro100_list)]

,product_id,cat1.0,cat2.0,cat3.0,cat4.0,cat5.0,cat7.0,cat8.0,cat9.0,cat10.0,...,cat34.0,pub0.0,pub1.0,pub2.0,pub3.0,pub4.0,pri0.0,pri1.0,pri2.0,pri3.0
236,55868083,0,0,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
246,55869065,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
340,55936772,0,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,0,0,1,0
348,55944631,0,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,0,0,1,0
374,55976113,0,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
269937,85030922,0,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
270155,85108883,0,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
270172,85109178,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
270304,85112006,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1


In [19]:
user_context_add = pd.merge(df_100,user_fin,on='account_id',how='left')
user_context_add

,account_id,product_id,price,purchase,book_cluster,user_cluster,gender_0.0,gender_1.0,g_age_0.0,g_age_1.0,...,most_pref_23.0,most_pref_25.0,most_pref_27.0,most_pref_29.0,most_pref_33.0,most_pref_34.0,new_pref_0,new_pref_1,address_is_0,address_is_1
0,113512,89940862,0.0,0.0,4.0,3.0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,1
1,113512,84434768,0.0,0.0,2.0,3.0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,1
2,113512,74393338,0.0,0.0,2.0,3.0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,1
3,113512,90108425,0.0,0.0,3.0,3.0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,1
4,113512,85927898,0.0,0.0,3.0,3.0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4128,16790454,90061659,0.0,0.0,3.0,1.0,0,1,0,0,...,0,0,0,0,0,0,0,1,1,0
4129,16790454,89222585,14400.0,1.0,3.0,1.0,0,1,0,0,...,0,0,0,0,0,0,0,1,1,0
4130,16790454,89903437,0.0,0.0,3.0,1.0,0,1,0,0,...,0,0,0,0,0,0,0,1,1,0
4131,16790454,90114596,0.0,0.0,3.0,1.0,0,1,0,0,...,0,0,0,0,0,0,0,1,1,0


In [20]:
df_100_fin = pd.merge(user_context_add, book_fin, on='product_id',how='left')
df_100_fin

,account_id,product_id,price,purchase,book_cluster,user_cluster,gender_0.0,gender_1.0,g_age_0.0,g_age_1.0,...,cat34.0,pub0.0,pub1.0,pub2.0,pub3.0,pub4.0,pri0.0,pri1.0,pri2.0,pri3.0
0,113512,89940862,0.0,0.0,4.0,3.0,1,0,0,0,...,0,0,0,0,0,1,0,0,1,0
1,113512,84434768,0.0,0.0,2.0,3.0,1,0,0,0,...,1,0,1,0,0,0,0,0,1,0
2,113512,74393338,0.0,0.0,2.0,3.0,1,0,0,0,...,1,0,0,1,0,0,0,0,1,0
3,113512,90108425,0.0,0.0,3.0,3.0,1,0,0,0,...,0,1,0,0,0,0,0,1,0,0
4,113512,85927898,0.0,0.0,3.0,3.0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4128,16790454,90061659,0.0,0.0,3.0,1.0,0,1,0,0,...,0,1,0,0,0,0,0,0,1,0
4129,16790454,89222585,14400.0,1.0,3.0,1.0,0,1,0,0,...,0,1,0,0,0,0,0,0,1,0
4130,16790454,89903437,0.0,0.0,3.0,1.0,0,1,0,0,...,0,1,0,0,0,0,0,0,1,0
4131,16790454,90114596,0.0,0.0,3.0,1.0,0,1,0,0,...,0,1,0,0,0,0,0,0,1,0


In [63]:
df_100_fin.to_csv('df100_test_0701.csv',index=False)